In [1]:
"""
Ethereum Price Prediction - Data Loading & Preprocessing
"""
# ============================================================================
# 기본 라이브러리 및 유틸리티
# ============================================================================


import gc
import time
import json
import joblib
import os
import warnings
from dotenv import load_dotenv
# 날짜/시간
from datetime import datetime, timedelta

# 데이터 처리
import numpy as np
import pandas as pd
import pandas_ta as ta
from collections import Counter
from numba import jit
# ============================================================================
# ML/DL 라이브러리 및 도구
# ============================================================================

# 하이퍼파라미터 최적화
import optuna

# Scikit-learn: 데이터 전처리
from sklearn.feature_selection import (
    SelectKBest, RFE,
    mutual_info_classif, mutual_info_regression
)
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler, StandardScaler

# Scikit-learn: 모델
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

# Scikit-learn: 앙상블 모델
from sklearn.ensemble import (
    AdaBoostClassifier, AdaBoostRegressor,
    BaggingClassifier, BaggingRegressor,
    ExtraTreesClassifier, ExtraTreesRegressor,
    GradientBoostingClassifier, GradientBoostingRegressor,
    HistGradientBoostingClassifier, 
    RandomForestClassifier, RandomForestRegressor,
    StackingClassifier, StackingRegressor,
    VotingClassifier, VotingRegressor
)

# Scikit-learn: 평가 지표
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, # 분류 지표
    mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error # 회귀 지표
)


from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from lightgbm.callback import early_stopping 
from xgboost import XGBClassifier, XGBRegressor

# TensorFlow/Keras 딥러닝
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    # 기본 레이어
    Input, Dense, Flatten, Dropout, Activation,
    # RNN 레이어
    LSTM, GRU, SimpleRNN, Bidirectional,
    # CNN 레이어
    Conv1D, MaxPooling1D, AveragePooling1D,
    GlobalAveragePooling1D, GlobalMaxPooling1D,
    # 정규화 레이어
    BatchNormalization, LayerNormalization,
    # Attention 레이어
    Attention, MultiHeadAttention,
    # 유틸리티 레이어
    Concatenate, Add, Multiply, Lambda,
    Reshape, Permute, RepeatVector, TimeDistributed
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# 시계열 분석 (Statsmodels)
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.vector_ar.var_model import VAR

# PyTorch 
try:
    import torch
    import torch.nn as nn
except ImportError:
    pass




import optuna
from sklearn.ensemble import (
    RandomForestClassifier, AdaBoostClassifier, BaggingClassifier,
    GradientBoostingClassifier, ExtraTreesClassifier, StackingClassifier,
    VotingClassifier, HistGradientBoostingClassifier
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from lightgbm.callback import early_stopping
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


2025-12-05 22:49:25.062846: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-05 22:49:25.062885: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-05 22:49:25.064238: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-05 22:49:25.071251: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-05 22:49:25.888361: W tensorflow/comp

In [2]:
# df_merged.head(10)
# macro_df.tail(20)

In [3]:


def handle_missing_values_paper_based(df_clean, train_start_date, is_train=True, train_stats=None):
    """
    암호화폐 시계열 결측치 처리
    
    참고문헌:
    1. "Quantifying Cryptocurrency Unpredictability" (2025)

    2. "Time Series Data Forecasting" 
    
    3. "Dealing with Leaky Missing Data in Production" (2021)

    """
    
    # ===== 1. Lookback 제거 =====
    if isinstance(train_start_date, str):
        train_start_date = pd.to_datetime(train_start_date)
    
    before = len(df_clean)
    df_clean = df_clean[df_clean['date'] >= train_start_date].reset_index(drop=True)
    
    # ===== 2. Feature 컬럼 선택 =====
    target_cols = ['next_log_return', 'next_direction', 'next_close','next_open','take_profit','']
    feature_cols = [col for col in df_clean.columns 
                   if col not in target_cols + ['date']]
    
    # ===== 3. 결측 확인 =====
    missing_before = df_clean[feature_cols].isnull().sum().sum()
    
    # ===== 4. FFill → 0 =====
    df_clean[feature_cols] = df_clean[feature_cols].fillna(method='ffill')
    df_clean[feature_cols] = df_clean[feature_cols].fillna(0)
    
    missing_after = df_clean[feature_cols].isnull().sum().sum()
    
    # ===== 5. 무한대 처리 =====
    inf_count = 0
    for col in feature_cols:
        if np.isinf(df_clean[col]).sum() > 0:
            inf_count += np.isinf(df_clean[col]).sum()
            df_clean[col] = df_clean[col].replace([np.inf, -np.inf], np.nan)
            df_clean[col] = df_clean[col].fillna(method='ffill').fillna(0)
    
    # ===== 6. 최종 확인 =====
    final_missing = df_clean[feature_cols].isnull().sum().sum()
    
    if final_missing > 0:
        df_clean[feature_cols] = df_clean[feature_cols].fillna(0)
    
    
    if is_train:
        return df_clean, {}
    else:
        return df_clean
    
       

In [4]:
import re

def remove_raw_price_columns(df):
    """
    비정상적 raw price 컬럼 제거
    (Feature 생성은 하지 않음)
    """
    df_clean = df.copy()
    timestamp = datetime.now().strftime("%Y-%m-%d")
    base_dir = os.path.join('model_results', timestamp, 'raw_data')
    os.makedirs(base_dir, exist_ok=True)
    df.to_csv(os.path.join(base_dir, "raw_data_all_features.csv"), 
                index=False)
    print(f"Saved raw data to {base_dir}")
    target_suffixes = ['_Open', '_High', '_Low', '_Close', '_Volume', '_Vol']
    
    cols_to_drop = []
    
    for col in df_clean.columns:
        is_raw_price = any(col.endswith(suffix) for suffix in target_suffixes)
        is_target = 'next_' in col.lower() or 'target' in col.lower()
        
        if is_raw_price and not is_target:
            cols_to_drop.append(col)
    
    # ATR_14 제거 (NATR_14가 있으므로)
    if 'ATR_14' in df_clean.columns:
        cols_to_drop.append('ATR_14')
    
    if cols_to_drop:
        print(f"Removing {len(cols_to_drop)} raw price columns")
        df_clean.drop(columns=cols_to_drop, inplace=True)
    
    return df_clean


In [5]:
import numpy as np
import pandas as pd
import pandas_ta as ta
from numba import jit


@jit(nopython=True)
def compute_triple_barrier_targets_next_open(
    prices_open,
    prices_high,
    prices_low,
    prices_close,
    atr,
    lookahead_candles,
    atr_multiplier_profit,
    atr_multiplier_stop
):
    n = len(prices_open)
    targets_raw = np.full(n, np.nan, dtype=np.float64) 
    take_profit_arr = np.zeros(n, dtype=np.float64) 
    stop_loss_arr = np.zeros(n, dtype=np.float64)
    
    max_i = n - lookahead_candles - 1
    
    for i in range(max_i):
        entry_price = prices_open[i + 1]
        current_atr = max(atr[i], 1e-8)
        
        upper_barrier = entry_price + (current_atr * atr_multiplier_profit)
        lower_barrier = entry_price - (current_atr * atr_multiplier_stop)
        
        take_profit_arr[i] = upper_barrier
        stop_loss_arr[i] = lower_barrier
        
        outcome = np.nan 
        
        for j in range(1, lookahead_candles + 1):
            future_idx = i + j
            future_high = prices_high[future_idx]
            future_low = prices_low[future_idx]
            
            hit_tp = future_high >= upper_barrier
            hit_sl = future_low <= lower_barrier
            
            if hit_sl and hit_tp:
                outcome = 0.0 
                break
            elif hit_sl:
                outcome = 0.0
                break
            elif hit_tp:
                outcome = 1.0
                break

        if np.isnan(outcome):
            exit_price = prices_close[i + lookahead_candles]
            if exit_price > entry_price + (current_atr * 0.5): 
                targets_raw[i] = 1.0
            else:
                targets_raw[i] = 0.0
        else:
            targets_raw[i] = outcome
    
    return targets_raw, take_profit_arr, stop_loss_arr


def create_targets(df, lookahead_candles=5, atr_multiplier_profit=2.0, atr_multiplier_stop=1.0):
    # 원본 훼손 방지
    df_target = df.copy()
    
    high = df_target['ETH_High']
    low = df_target['ETH_Low']
    close = df_target['ETH_Close']
    atr_col_name = 'ATR_14'

    # 1. ATR 계산
    atr_series = ta.atr(high, low, close, length=14)

    if atr_series is None:
        tr = high - low
        atr_series = tr.rolling(14).mean()

    df_target[atr_col_name] = atr_series.fillna(method='ffill').fillna(0.0)
    
    # -------------------------------------------------------
    
    # Numpy 변환
    prices_open = df_target['ETH_Open'].to_numpy(dtype=np.float64)
    prices_high = df_target['ETH_High'].to_numpy(dtype=np.float64)
    prices_low = df_target['ETH_Low'].to_numpy(dtype=np.float64)
    prices_close=df_target['ETH_Close'].to_numpy(dtype=np.float64)
    atr = df_target[atr_col_name].to_numpy(dtype=np.float64)

    # Triple Barrier 계산
    targets_raw, tp_prices, sl_prices = compute_triple_barrier_targets_next_open(
        prices_open,
        prices_high,
        prices_low,
        prices_close,
        atr,
        lookahead_candles,
        atr_multiplier_profit,
        atr_multiplier_stop
    )

    
    # 결과 할당
    df_target['next_direction'] = targets_raw
    df_target['take_profit_price'] = tp_prices
    df_target['stop_loss_price'] = sl_prices
    
    # 시각화용 처리
    df_target['take_profit_price'] = df_target['take_profit_price'].replace(0, np.nan)
    df_target['stop_loss_price'] = df_target['stop_loss_price'].replace(0, np.nan)

    # 파생 변수 생성 (에러 방지용)
    df_target['next_close'] = df_target['ETH_Close'].shift(-1)
    df_target['next_open'] = df_target['ETH_Open'].shift(-1)
    df_target['next_log_return'] = np.log(df_target['next_close'] / (df_target['next_open'] + 1e-9))
    balance = df_target['next_direction'].value_counts(normalize=True)
    print("=== 정답(Label) 분포 확인 ===")
    print(balance)
    return df_target


In [6]:
import numpy as np
import pandas as pd
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import squareform
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression, RFE
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from collections import Counter

# ==============================================================================
# [수정 완료] 안전한 피처 선택 함수 (부동소수점 에러 패치 + 3-Way 앙상블)
# ==============================================================================
def select_features_verified(X_train, y_train, task='class', top_n=30, verbose=True):
    """
    1. 계층적 클러스터링으로 다중공선성 제거 (음수 거리 에러 방지)
    2. MI + RFE + RF 3-Way Voting으로 최종 피처 선정
    """
    # --- [Patch] 안전한 거리 행렬 계산 ---
    # 1. 상관관계 계산 및 NaN 처리
    corr_matrix = X_train.corr().abs().fillna(0)
    
    # 2. 거리 변환 (1 - corr)
    dist_matrix = 1 - corr_matrix
    
    # 3. [핵심] 부동소수점 오차로 인한 음수값(-0.000...)을 0으로 강제 치환
    # 이 부분이 없으면 'ValueError: Linkage contains negative distances' 발생
    dist_matrix = dist_matrix.clip(lower=0)
    
    # 4. 대각선(자기 자신과의 거리)을 0으로 명시적 설정
    np.fill_diagonal(dist_matrix.values, 0)
    
    # 5. squareform 변환 (checks=False로 미세 오차 무시)
    condensed_dist = squareform(dist_matrix, checks=False)
    
    # 6. 계층적 클러스터링 수행 (Ward 연결법)
    # Ward가 실패할 경우를 대비한 try-except가 있으면 좋지만, 
    # 위에서 clip(lower=0)을 했으므로 Ward도 안전하게 돌아감
    linkage_matrix = linkage(condensed_dist, method='ward')
    
    # 7. 클러스터링 (t=0.3: 상관관계 0.7 이상끼리 묶음)
    cluster_labels = fcluster(linkage_matrix, t=0.3, criterion='distance')
    
    # --- [Feature Scoring] MI 점수 계산 ---
    if task == 'class':
        mi_scores_all = mutual_info_classif(X_train, y_train, random_state=42, n_neighbors=3)
    else: # regression
        mi_scores_all = mutual_info_regression(X_train, y_train, random_state=42, n_neighbors=3)
        
    mi_score_dict_all = dict(zip(X_train.columns, mi_scores_all))
    
    # --- [Compression] 클러스터별 대표 피처 선정 ---
    unique_features = []
    for cluster_id in np.unique(cluster_labels):
        members = X_train.columns[cluster_labels == cluster_id]
        # 해당 클러스터 멤버 중 MI 점수가 가장 높은 1개만 선택 (대표)
        best_feature = max(members, key=lambda x: mi_score_dict_all[x])
        unique_features.append(best_feature)
        
    # 중복이 제거된 데이터셋 생성
    X_reduced = X_train[unique_features].copy()
    
    if verbose:
        print(f"[Clustering] {len(X_train.columns)}개 -> {len(unique_features)}개로 압축 (중복 제거 완료)")

    # =========================================================
    # [Step 1] 3-Way Ensemble Selection (대상: X_reduced)
    # =========================================================
    
    # 1. MI (Mutual Information) Top-N
    reduced_mi_scores = {k: v for k, v in mi_score_dict_all.items() if k in unique_features}
    mi_selected = sorted(reduced_mi_scores, key=reduced_mi_scores.get, reverse=True)[:top_n]
    
    # 모델 설정 (분류 vs 회귀)
    if task == 'class':
        lgbm_model = LGBMClassifier(n_estimators=100, learning_rate=0.05, max_depth=5, random_state=42, verbose=-1)
        rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
    else:
        lgbm_model = LGBMRegressor(n_estimators=100, learning_rate=0.05, max_depth=5, random_state=42, verbose=-1)
        rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)

    # 2. RFE (Recursive Feature Elimination)
    # step=0.1: 한 번에 10%씩 제거하여 속도 향상
    rfe = RFE(estimator=lgbm_model, n_features_to_select=top_n, step=0.1, verbose=0)
    rfe.fit(X_reduced, y_train)
    rfe_features = X_reduced.columns[rfe.support_].tolist()

    # 3. RF (Random Forest Importance)
    rf_model.fit(X_reduced, y_train)
    rf_importances = rf_model.feature_importances_
    
    # 중요도 순으로 정렬하여 상위 N개 선택
    rf_idx = np.argsort(rf_importances)[::-1][:top_n]
    rf_features = X_reduced.columns[rf_idx].tolist()
    
    # =========================================================
    # [Step 2] Final Voting (Majority Rule)
    # =========================================================
    all_candidates = mi_selected + rfe_features + rf_features
    feature_votes = Counter(all_candidates)
    
    # 2표 이상 받은 피처 우선 선발 (우등생)
    selected_features = [feat for feat, count in feature_votes.most_common() if count >= 2]

    # 부족하면 1표 받은 애들 중 MI 점수 높은 순으로 채움 (추가합격)
    if len(selected_features) < top_n:
        remaining = top_n - len(selected_features)
        others = [feat for feat, count in feature_votes.most_common() if count == 1]
        # 전체 MI 점수 사전에서 점수 가져와 정렬
        others_sorted = sorted(others, key=lambda x: mi_score_dict_all[x], reverse=True)
        
        for feat in others_sorted:
            if feat not in selected_features:
                selected_features.append(feat)
                remaining -= 1
                if remaining == 0:
                    break
    
    # 최종적으로 top_n개로 자름 (혹시 모를 초과 방지)
    selected_features = selected_features[:top_n]
    
    return selected_features, {
        'mi_features': mi_selected,
        'rfe_features': rfe_features,
        'rf_features': rf_features,
        'feature_votes': feature_votes,
        'clustering_survivors': unique_features
    }


def select_features_multi_target(X_train, y_train, target_type='direction', top_n=30):
    """
    메인 호출 함수: 타겟 타입에 따라 적절한 모드로 select_features_verified 호출
    """
    atr_col_name = 'NATR_14'

    # 분류(Direction) vs 회귀(Price) 분기 처리
    if target_type == 'direction':
        task_type = 'class'
        # y_train이 DataFrame이면 시리즈로 변환
        target_y = y_train['next_direction'] if isinstance(y_train, pd.DataFrame) else y_train
    else:
        task_type = 'reg'
        target_y = y_train # 회귀면 그냥 y_train 사용

    # 검증된 함수 호출
    selected, stats = select_features_verified(
        X_train, 
        target_y, 
        task=task_type, 
        top_n=top_n
    )
    
    # NATR_14(변동성 지표) 강제 포함 로직 (없으면 추가)
    if atr_col_name not in selected and atr_col_name in X_train.columns:
        # 마지막(중요도 꼴찌) 하나 빼고 ATR 넣음
        if len(selected) >= top_n:
            selected.pop() 
        selected.insert(0, atr_col_name) 
    
    print(f"최종 선택된 피처({len(selected)}개): {', '.join(selected)}")
    return selected, stats



In [7]:
def split_walk_forward_method(df, train_start_date, 
                              final_test_start='2025-01-01',
                              initial_train_size=6600,    
                              val_size=90,             
                              test_size=50,           
                              step=50,                  
                              gap_size=8):

    df_period = df[df['date'] >= train_start_date].copy()
    df_period = df_period.sort_values('date').reset_index(drop=True)
    
    if isinstance(final_test_start, str):
        final_test_start = pd.to_datetime(final_test_start)
    
    final_test_df = df_period[df_period['date'] >= final_test_start].copy()
    
    total_days = len(df_period)
    min_required_days = initial_train_size + val_size + (gap_size * 2) + test_size
    
    if total_days < min_required_days:
        print(f"Not enough data. Total: {total_days}, Required: {min_required_days}")
        n_splits = 0
    else:
        n_splits = (total_days - min_required_days) // step + 1
    
    print(f"\n{'='*80}")
    print(f"Reverse Rolling Walk-Forward Configuration ")
    print(f"{'='*80}")
    print(f"Total: {len(df_period)} days")
    print(f"Rolling train size: {initial_train_size} days (FIXED)")
    print(f"Val: {val_size} days | Test: {test_size} days")
    print(f"Gap: {gap_size} days | Step: {step} days (BACKWARD)")
    print(f"Target: {n_splits} walk-forward + 1 final holdout")
    print(f"{'='*80}\n")
    
    folds = []
    
    # 역방향 rolling
    for fold_idx in range(n_splits):
        test_end_idx = total_days - (fold_idx * step)
        test_start_idx = test_end_idx - test_size
        
        if test_start_idx < 0:
            break
        
        val_end_idx = test_start_idx - gap_size
        val_start_idx = val_end_idx - val_size
        
        train_end_idx = val_start_idx - gap_size
        train_start_idx = train_end_idx - initial_train_size
        
        if train_start_idx < 0:
            break
        
        train_fold = df_period.iloc[train_start_idx:train_end_idx].copy()
        val_fold = df_period.iloc[val_start_idx:val_end_idx].copy()
        test_fold = df_period.iloc[test_start_idx:test_end_idx].copy()
        
        folds.append({
            'train': train_fold,
            'val': val_fold,
            'test': test_fold,
            'fold_idx': fold_idx + 1,
            'fold_type': 'walk_forward_rolling_reverse'
        })
    
    # 시간순으로 정렬
    folds.reverse()
    for idx, fold in enumerate(folds):
        fold['fold_idx'] = idx + 1
        
        print(f"Fold {fold['fold_idx']} (walk_forward_rolling)")
        print(f"  Train: {len(fold['train']):4d}d  {fold['train']['date'].min().date()} ~ {fold['train']['date'].max().date()}")
        print(f"  Val:   {len(fold['val']):4d}d  {fold['val']['date'].min().date()} ~ {fold['val']['date'].max().date()}")
        print(f"  Test:  {len(fold['test']):4d}d  {fold['test']['date'].min().date()} ~ {fold['test']['date'].max().date()}\n")
    
    # Final holdout
    if len(final_test_df) > 0:
        pre_final_df = df_period[df_period['date'] < final_test_start].copy()
        
        final_val_end_idx = len(pre_final_df)
        final_val_start_idx = final_val_end_idx - val_size
        final_train_end_idx = final_val_start_idx - gap_size
        final_train_start_idx = final_train_end_idx - initial_train_size
        
        if final_train_start_idx < 0:
            final_train_start_idx = 0
        
        final_train_data = pre_final_df.iloc[final_train_start_idx:final_train_end_idx].copy()
        final_val_data = pre_final_df.iloc[final_val_start_idx:final_val_end_idx].copy()
        
        print(f"Fold {len(folds) + 1} (final_holdout)")
        print(f"  Train: {len(final_train_data):4d}d  {final_train_data['date'].min().date()} ~ {final_train_data['date'].max().date()}")
        print(f"  Val:   {len(final_val_data):4d}d  {final_val_data['date'].min().date()} ~ {final_val_data['date'].max().date()}")
        print(f"  Test:  {len(final_test_df):4d}d  {final_test_df['date'].min().date()} ~ {final_test_df['date'].max().date()}\n")
        
        folds.append({
            'train': final_train_data,
            'val': final_val_data,
            'test': final_test_df,
            'fold_idx': len(folds) + 1,
            'fold_type': 'final_holdout'
        })
    
    print(f"{'='*80}")
    print(f"Created {len(folds)} folds total")
    print(f"{'='*80}\n")
    
    return folds


def process_single_split(split_data, target_type='direction', top_n=30, fold_idx=None):
    """
    각 fold를 독립적으로 처리 (feature selection 포함)
    """
    
    train_df = split_data['train'].copy()
    val_df = split_data['val'].copy()
    test_df = split_data['test'].copy()
    fold_type = split_data.get('fold_type', 'unknown')
    
    if fold_idx is not None:
        print(f"\n{'='*60}")
        print(f"Processing Fold {fold_idx} ({fold_type})")
        print(f"{'='*60}")
    
    target_cols = ['next_direction','next_log_return', 'next_close','next_open', 
                   'take_profit_price', 'stop_loss_price']

    len_before_train = len(train_df)
    train_df = train_df.dropna(subset=target_cols).reset_index(drop=True)
    
    len_before_val = len(val_df)
    val_df = val_df.dropna(subset=target_cols).reset_index(drop=True)
    
    len_before_test = len(test_df)
    test_df = test_df.dropna(subset=target_cols).reset_index(drop=True)
    
    # 제거 로그 출력
    print(f"[Pre-cleaning] Dropped rows with NaN targets (Leakage prevention):")
    print(f"  Train: {len_before_train} -> {len(train_df)} (Dropped {len_before_train - len(train_df)})")
    print(f"  Val:   {len_before_val} -> {len(val_df)} (Dropped {len_before_val - len(val_df)})")
    print(f"  Test:  {len_before_test} -> {len(test_df)} (Dropped {len_before_test - len(test_df)})")

    # [Step 2] 결측치 처리
    train_processed, missing_stats = handle_missing_values_paper_based(
        train_df,
        train_start_date=train_df['date'].min(),
        is_train=True
    )
    
    val_processed = handle_missing_values_paper_based(
        val_df,
        train_start_date=val_df['date'].min(),
        is_train=False,
        train_stats=missing_stats
    )
    
    test_processed = handle_missing_values_paper_based(
        test_df,
        train_start_date=test_df['date'].min(),
        is_train=False,
        train_stats=missing_stats
    )
    
    # Feature 컬럼 정의
    feature_cols = [col for col in train_processed.columns 
                   if col not in target_cols + ['date']]
    
    # X, y 분리
    X_train = train_processed[feature_cols]
    y_train = train_processed[target_cols]
    
    X_val = val_processed[feature_cols]
    y_val = val_processed[target_cols]
    
    X_test = test_processed[feature_cols]
    y_test = test_processed[target_cols]

    # [Step 3] Feature Selection
    print(f"\n[Feature Selection for Fold {fold_idx}]")
    print(f"Training data shape: {X_train.shape}")
    
    selected_features, selection_stats = select_features_multi_target(
        X_train, 
        y_train, 
        target_type=target_type, 
        top_n=top_n
    )
    
    print(f"Selected {len(selected_features)} features for this fold")
    
    # 선택된 피처만 필터링
    X_train_sel = X_train[selected_features]
    X_val_sel = X_val[selected_features]
    X_test_sel = X_test[selected_features]
    
    X_train_robust = X_train_sel.values
    X_val_robust   = X_val_sel.values
    X_test_robust  = X_test_sel.values
    
    # Standard 변수명도 뒤에서 쓰신다면, 똑같이 값을 넣어주면 됩니다. (에러 방지용)
    X_train_standard = X_train_sel.values
    X_val_standard   = X_val_sel.values
    X_test_standard  = X_test_sel.values
    
    print(f"Scaling completed for Fold {fold_idx}")
    print(f"{'='*60}\n")
    
    # 최종 데이터 확인 로그
    print(f"\n--- [Verification] Actual Date Ranges for Fold {fold_idx} (Final) ---")
    if not train_processed.empty:
        print(f"  Train: {len(train_processed):4d} ({train_processed['date'].min().date()} ~ {train_processed['date'].max().date()})")
    if not val_processed.empty:
        print(f"  Val:   {len(val_processed):4d} ({val_processed['date'].min().date()} ~ {val_processed['date'].max().date()})")
    if not test_processed.empty:
        print(f"  Test:  {len(test_processed):4d} ({test_processed['date'].min().date()} ~ {test_processed['date'].max().date()})")
    print(f"---------------------------------------------------------------------\n")
    
    result = {
        'train': {
            'X_robust': X_train_robust,
            'X_standard': X_train_standard,
            'X_raw': X_train_sel,
            'y': y_train.reset_index(drop=True), 
            'dates': train_processed['date'].reset_index(drop=True) 
        },
        'val': {
            'X_robust': X_val_robust,
            'X_standard': X_val_standard,
            'X_raw': X_val_sel,
            'y': y_val.reset_index(drop=True), 
            'dates': val_processed['date'].reset_index(drop=True)
        },
        'test': {
            'X_robust': X_test_robust,
            'X_standard': X_test_standard,
            'X_raw': X_test_sel,
            'y': y_test.reset_index(drop=True),
            'dates': test_processed['date'].reset_index(drop=True)
        },
        'stats': {

            'selected_features': selected_features,
            'selection_stats': selection_stats,
            'missing_stats': missing_stats,  
            'target_type': target_type,
            'target_cols': target_cols,
            'fold_type': fold_type,
            'fold_idx': fold_idx
        }
    }
    
    return result


def build_complete_pipeline_corrected(df_raw, train_start_date, 
                                     final_test_start='2025-01-01',
                                     method='tvt', target_type='direction',lookahead_candles=10,atr_multiplier_profit=1.5, 
                                     atr_multiplier_stop=1.0, **kwargs):

    original_last_date = pd.to_datetime(df_raw['date']).max()
    
    df = df_raw.copy()

    df = create_targets(df, 
                        lookahead_candles=lookahead_candles,
                        atr_multiplier_profit=atr_multiplier_profit, 
                        atr_multiplier_stop=atr_multiplier_stop)
    df = remove_raw_price_columns(df)

    current_last_date = pd.to_datetime(df['date']).max()
    

    if current_last_date == original_last_date:
        df = df.iloc[:-lookahead_candles]
    else:
        print(f"Already Cleaned: Last date changed ({original_last_date.date()} -> {current_last_date.date()})")
    
    
    split_kwargs = {}
    split_kwargs['final_test_start'] = final_test_start
    if 'n_splits' in kwargs:
        split_kwargs['n_splits'] = kwargs['n_splits']
    if 'initial_train_size' in kwargs:
        split_kwargs['initial_train_size'] = kwargs['initial_train_size']
    if 'test_size' in kwargs:
        split_kwargs['test_size'] = kwargs['test_size']
    if 'val_size' in kwargs:
        split_kwargs['val_size'] = kwargs['val_size']
    if 'step' in kwargs:
        split_kwargs['step'] = kwargs['step']
        
    splits = split_walk_forward_method(df, train_start_date, **split_kwargs)

    result = [
        process_single_split(
            fold, 
            target_type=target_type,  
            top_n=30,
            fold_idx=fold['fold_idx']
        ) 
        for fold in splits
    ]
    
    return result



In [8]:
import numpy as np
import pandas as pd
import optuna
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Bidirectional, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import accuracy_score
import tensorflow as tf



In [9]:

import numpy as np
import tensorflow as tf
import optuna
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, f1_score  # [핵심] 평가 지표 변경
from sklearn.utils.class_weight import compute_sample_weight
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional, GRU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

class DirectionModels:
    
    # [Helper] 시간 흐름에 따른 가중치 + 클래스 불균형 가중치 통합 계산
    @staticmethod
    def _get_combined_weights(y_train, decay_start=0.4):
        # 1. 시간 가중치 (과거 -> 최신: 0.4 -> 1.0 선형 증가)
        time_weights = np.linspace(decay_start, 1.0, len(y_train))
        
        # 2. 클래스 불균형 가중치 (희소 클래스에 가중치)
        class_weights = compute_sample_weight('balanced', y_train)
        
        # 3. 두 가중치 결합 (곱하기)
        combined_weights = time_weights * class_weights
        return combined_weights

    @staticmethod
    def random_forest(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        weights = DirectionModels._get_combined_weights(y_train)
        
        def objective(trial):
            param = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 200),
                'max_depth': trial.suggest_int('max_depth', 3, 6),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 20, 50),
                'max_features': 'sqrt',
                'random_state': 42,
                'n_jobs': -1
            }
            
            model = RandomForestClassifier(**param)
            model.fit(X_train, y_train, sample_weight=weights)
            
            # [수정] Accuracy -> ROC AUC
            # 확률값(predict_proba)을 기준으로 1을 얼마나 잘 골라내는지 평가
            y_pred_proba = model.predict_proba(X_val)[:, 1]
            val_auc = roc_auc_score(y_val, y_pred_proba)
            return val_auc

        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
        study.optimize(objective, n_trials=20, show_progress_bar=False)
        
        best_model = RandomForestClassifier(**study.best_params, n_jobs=-1, random_state=42)
        best_model.fit(X_train, y_train, sample_weight=weights)
        
        val_auc = roc_auc_score(y_val, best_model.predict_proba(X_val)[:, 1])
        print(f"[RandomForest] Val AUC: {val_auc:.4f}")
        return best_model
    
    @staticmethod
    def lightgbm(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        weights = DirectionModels._get_combined_weights(y_train)

        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 200),
                'max_depth': trial.suggest_int('max_depth', 2, 5),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05),
                'num_leaves': trial.suggest_int('num_leaves', 10, 25),
                'reg_alpha': trial.suggest_float('reg_alpha', 1.0, 10.0),
                'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 10.0),
                'min_child_samples': trial.suggest_int('min_child_samples', 20, 50),
                'random_state': 42,
                'n_jobs': -1,
                'verbose': -1
            }

            model = LGBMClassifier(**params)
            model.fit(
                X_train, y_train, 
                sample_weight=weights, 
                eval_set=[(X_val, y_val)]
            )
            
            # [수정] Accuracy -> ROC AUC
            y_pred_proba = model.predict_proba(X_val)[:, 1]
            return roc_auc_score(y_val, y_pred_proba)

        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
        study.optimize(objective, n_trials=20, show_progress_bar=False)

        best_params = study.best_params
        best_params.update({'random_state': 42, 'n_jobs': -1, 'verbose': -1})
        
        final_model = LGBMClassifier(**best_params)
        final_model.fit(
            X_train, y_train, 
            sample_weight=weights, 
            eval_set=[(X_val, y_val)]
        )
        
        val_auc = roc_auc_score(y_val, final_model.predict_proba(X_val)[:, 1])
        print(f"[LightGBM] Val AUC: {val_auc:.4f}")
        return final_model

    @staticmethod
    def xgboost(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        weights = DirectionModels._get_combined_weights(y_train)

        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 200),
                'max_depth': trial.suggest_int('max_depth', 2, 5),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05),
                'subsample': 0.7,
                'reg_alpha': trial.suggest_float('reg_alpha', 1.0, 10.0),
                'min_child_weight': trial.suggest_int('min_child_weight', 5, 20),
                'random_state': 42,
                'n_jobs': -1,
                'tree_method': 'hist'
            }

            model = XGBClassifier(**params)
            model.fit(X_train, y_train, sample_weight=weights, eval_set=[(X_val, y_val)], verbose=False)
            
            # [수정] Accuracy -> ROC AUC
            y_pred_proba = model.predict_proba(X_val)[:, 1]
            return roc_auc_score(y_val, y_pred_proba)

        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
        study.optimize(objective, n_trials=20, show_progress_bar=False)

        best_params = study.best_params
        best_params.update({'random_state': 42, 'n_jobs': -1, 'tree_method': 'hist'})

        final_model = XGBClassifier(**best_params)
        final_model.fit(X_train, y_train, sample_weight=weights, eval_set=[(X_val, y_val)], verbose=False)
        
        val_auc = roc_auc_score(y_val, final_model.predict_proba(X_val)[:, 1])
        print(f"[XGBoost] Val AUC: {val_auc:.4f}")
        return final_model

    @staticmethod
    def histgradient_boosting(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        weights = DirectionModels._get_combined_weights(y_train)
        
        def objective(trial):
            params = {
                'max_iter': trial.suggest_int('max_iter', 100, 200),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05),
                'max_depth': trial.suggest_int('max_depth', 2, 5),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 20, 50),
                'l2_regularization': trial.suggest_float('l2_regularization', 1.0, 10.0),
                'random_state': 42
            }
            
            model = HistGradientBoostingClassifier(**params)
            model.fit(X_train, y_train, sample_weight=weights)
            
            # [수정] Accuracy -> ROC AUC
            y_pred_proba = model.predict_proba(X_val)[:, 1]
            return roc_auc_score(y_val, y_pred_proba)
        
        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
        study.optimize(objective, n_trials=20, show_progress_bar=False)
        
        best_model = HistGradientBoostingClassifier(**study.best_params, random_state=42)
        best_model.fit(X_train, y_train, sample_weight=weights)
        
        val_auc = roc_auc_score(y_val, best_model.predict_proba(X_val)[:, 1])
        print(f"[HistGradientBoosting] Val AUC: {val_auc:.4f}")
        return best_model

    @staticmethod
    def logistic_regression(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        weights = DirectionModels._get_combined_weights(y_train)
        
        def objective(trial):
            param = {
                'C': trial.suggest_float('C', 0.001, 0.5, log=True),
                'solver': 'lbfgs',
                'max_iter': 500,
                'random_state': 42,
                'n_jobs': -1
            }
            model = LogisticRegression(**param)
            model.fit(X_train, y_train, sample_weight=weights)
            
            # [수정] Accuracy -> ROC AUC
            y_pred_proba = model.predict_proba(X_val)[:, 1]
            return roc_auc_score(y_val, y_pred_proba)
        
        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
        study.optimize(objective, n_trials=15, show_progress_bar=False)
        
        best_model = LogisticRegression(**study.best_params, max_iter=1000, n_jobs=-1, random_state=42)
        best_model.fit(X_train, y_train, sample_weight=weights)
        
        val_auc = roc_auc_score(y_val, best_model.predict_proba(X_val)[:, 1])
        print(f"[Logistic Regression] Val AUC: {val_auc:.4f}")
        return best_model

    @staticmethod
    def adaboost(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        weights = DirectionModels._get_combined_weights(y_train)
        
        def objective(trial):
            base = DecisionTreeClassifier(max_depth=2, max_features='sqrt')
            param = {
                'n_estimators': trial.suggest_int('n_estimators', 30, 80),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
                'algorithm': 'SAMME',
                'random_state': 42
            }
            model = AdaBoostClassifier(estimator=base, **param)
            model.fit(X_train, y_train, sample_weight=weights)
            
            # [수정] Accuracy -> ROC AUC
            y_pred_proba = model.predict_proba(X_val)[:, 1]
            return roc_auc_score(y_val, y_pred_proba)
        
        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
        study.optimize(objective, n_trials=20, show_progress_bar=False)
        
        final = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=2, max_features='sqrt'),
                                   **study.best_params, algorithm='SAMME', random_state=42)
        final.fit(X_train, y_train, sample_weight=weights)
        
        val_auc = roc_auc_score(y_val, final.predict_proba(X_val)[:, 1])
        print(f"[AdaBoost] Val AUC: {val_auc:.4f}")
        return final
    
    @staticmethod
    def catboost(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        weights = DirectionModels._get_combined_weights(y_train)
        
        def objective(trial):
            param = {
                'iterations': trial.suggest_int('iterations', 100, 200),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05),
                'depth': trial.suggest_int('depth', 2, 5),
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 3.0, 10.0),
                'random_seed': 42,
                'verbose': False,
                'allow_writing_files': False
            }
            model = CatBoostClassifier(**param)
            model.fit(X_train, y_train, sample_weight=weights, eval_set=(X_val, y_val), verbose=False)
            
            # [수정] Accuracy -> ROC AUC
            y_pred_proba = model.predict_proba(X_val)[:, 1]
            return roc_auc_score(y_val, y_pred_proba)
        
        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
        study.optimize(objective, n_trials=20, show_progress_bar=False)
        
        model = CatBoostClassifier(**study.best_params, random_seed=42, verbose=False, allow_writing_files=False)
        model.fit(X_train, y_train, sample_weight=weights, eval_set=(X_val, y_val), verbose=False)
        
        val_auc = roc_auc_score(y_val, model.predict_proba(X_val)[:, 1])
        print(f"[CatBoost] Val AUC: {val_auc:.4f}")
        return model

    @staticmethod
    def gradient_boosting(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        weights = DirectionModels._get_combined_weights(y_train)
        
        def objective(trial):
            param = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 200),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05),
                'max_depth': trial.suggest_int('max_depth', 2, 4),
                'subsample': 0.7,
                'max_features': 'sqrt',
                'validation_fraction': 0.15,
                'n_iter_no_change': 10,
                'random_state': 42
            }
            model = GradientBoostingClassifier(**param)
            model.fit(X_train, y_train, sample_weight=weights)
            
            # [수정] Accuracy -> ROC AUC
            y_pred_proba = model.predict_proba(X_val)[:, 1]
            return roc_auc_score(y_val, y_pred_proba)
        
        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
        study.optimize(objective, n_trials=20, show_progress_bar=False)
        
        best_model = GradientBoostingClassifier(**study.best_params, random_state=42)
        best_model.fit(X_train, y_train, sample_weight=weights)
        
        val_auc = roc_auc_score(y_val, best_model.predict_proba(X_val)[:, 1])
        print(f"[GradientBoosting] Val AUC: {val_auc:.4f}")
        return best_model

    @staticmethod
    def stacking_ensemble(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        base_learners = [
            ('xgb', XGBClassifier(n_estimators=150, max_depth=3, learning_rate=0.03, tree_method='hist', n_jobs=-1, random_state=42)),
            ('lgbm', LGBMClassifier(n_estimators=150, max_depth=3, learning_rate=0.03, verbose=-1, n_jobs=-1, random_state=42))
        ]
        meta_learner = LogisticRegression(C=0.1, max_iter=1000, random_state=42)
        
        model = StackingClassifier(estimators=base_learners, final_estimator=meta_learner, cv=3, n_jobs=-1)
        model.fit(X_train, y_train)
        
        # [수정] 평가 시 AUC 출력
        y_pred_proba = model.predict_proba(X_val)[:, 1]
        val_auc = roc_auc_score(y_val, y_pred_proba)
        print(f"[Stacking] Val AUC: {val_auc:.4f}")
        return model

    # ==========================================================================
    # [딥러닝 모델] - Evaluate(Loss, Acc) 대신 Predict -> AUC로 변경
    # ==========================================================================
    @staticmethod
    def lstm(X_train, y_train, X_val, y_val, input_shape):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        weights = DirectionModels._get_combined_weights(y_train) 

        def objective(trial):
            units = trial.suggest_int('units', 32, 128, step=32)
            dropout = trial.suggest_float('dropout', 0.2, 0.5)
            lr = trial.suggest_float('learning_rate', 1e-4, 1e-3, log=True)
            l2_val = trial.suggest_float('l2_reg', 1e-5, 1e-3, log=True)

            model = Sequential([
                LSTM(units, return_sequences=False, input_shape=input_shape,
                     kernel_regularizer=l2(l2_val), dropout=dropout),
                BatchNormalization(),
                Dense(16, activation='relu', kernel_regularizer=l2(l2_val)),
                Dropout(dropout),
                Dense(1, activation='sigmoid')
            ])

            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                          loss='binary_crossentropy', metrics=['accuracy'])
            
            early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
            
            model.fit(X_train, y_train, validation_data=(X_val, y_val),
                      epochs=30, batch_size=256, sample_weight=weights,
                      callbacks=[early_stop], verbose=0)
            
            # [수정] Accuracy -> ROC AUC (딥러닝은 predict하면 바로 확률값 나옴)
            y_pred_proba = model.predict(X_val, verbose=0).flatten()
            return roc_auc_score(y_val, y_pred_proba)

        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
        study.optimize(objective, n_trials=5, show_progress_bar=False)

        p = study.best_params
        model = Sequential([
            LSTM(p['units'], return_sequences=False, input_shape=input_shape,
                 kernel_regularizer=l2(p['l2_reg']), dropout=p['dropout']),
            BatchNormalization(),
            Dense(16, activation='relu', kernel_regularizer=l2(p['l2_reg'])),
            Dropout(p['dropout']),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=p['learning_rate']),
                      loss='binary_crossentropy', metrics=['accuracy'])
        
        model.fit(X_train, y_train, validation_data=(X_val, y_val),
                  epochs=50, batch_size=256, sample_weight=weights,
                  callbacks=[EarlyStopping(patience=10, restore_best_weights=True)], verbose=0)
        
        # 평가
        y_pred_proba = model.predict(X_val, verbose=0).flatten()
        val_auc = roc_auc_score(y_val, y_pred_proba)
        print(f"[LSTM] Val AUC: {val_auc:.4f}")
        return model

    @staticmethod
    def bilstm(X_train, y_train, X_val, y_val, input_shape):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        weights = DirectionModels._get_combined_weights(y_train)

        def objective(trial):
            units = trial.suggest_int('units', 32, 64, step=16)
            dropout = trial.suggest_float('dropout', 0.2, 0.5)
            lr = trial.suggest_float('learning_rate', 1e-4, 1e-3, log=True)
            l2_val = trial.suggest_float('l2_reg', 1e-5, 1e-3, log=True)

            model = Sequential([
                Bidirectional(LSTM(units, return_sequences=False,
                                   kernel_regularizer=l2(l2_val), dropout=dropout), input_shape=input_shape),
                BatchNormalization(),
                Dense(16, activation='relu', kernel_regularizer=l2(l2_val)),
                Dropout(dropout),
                Dense(1, activation='sigmoid')
            ])
            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                          loss='binary_crossentropy', metrics=['accuracy'])
            
            model.fit(X_train, y_train, validation_data=(X_val, y_val),
                      epochs=30, batch_size=256, sample_weight=weights,
                      callbacks=[EarlyStopping(patience=5, restore_best_weights=True)], verbose=0)
            
            # [수정] Accuracy -> ROC AUC
            y_pred_proba = model.predict(X_val, verbose=0).flatten()
            return roc_auc_score(y_val, y_pred_proba)

        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
        study.optimize(objective, n_trials=5, show_progress_bar=False)

        p = study.best_params
        model = Sequential([
            Bidirectional(LSTM(p['units'], return_sequences=False,
                               kernel_regularizer=l2(p['l2_reg']), dropout=p['dropout']), input_shape=input_shape),
            BatchNormalization(),
            Dense(16, activation='relu', kernel_regularizer=l2(p['l2_reg'])),
            Dropout(p['dropout']),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=p['learning_rate']),
                      loss='binary_crossentropy', metrics=['accuracy'])
        
        model.fit(X_train, y_train, validation_data=(X_val, y_val),
                  epochs=50, batch_size=256, sample_weight=weights,
                  callbacks=[EarlyStopping(patience=10, restore_best_weights=True)], verbose=0)
        
        y_pred_proba = model.predict(X_val, verbose=0).flatten()
        val_auc = roc_auc_score(y_val, y_pred_proba)
        print(f"[BiLSTM] Val AUC: {val_auc:.4f}")
        return model

    @staticmethod
    def gru(X_train, y_train, X_val, y_val, input_shape):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        weights = DirectionModels._get_combined_weights(y_train)

        def objective(trial):
            units = trial.suggest_int('units', 32, 128, step=32)
            dropout = trial.suggest_float('dropout', 0.2, 0.5)
            lr = trial.suggest_float('learning_rate', 1e-4, 1e-3, log=True)
            l2_val = trial.suggest_float('l2_reg', 1e-5, 1e-3, log=True)

            model = Sequential([
                GRU(units, return_sequences=False, input_shape=input_shape,
                    kernel_regularizer=l2(l2_val), dropout=dropout),
                BatchNormalization(),
                Dense(16, activation='relu', kernel_regularizer=l2(l2_val)),
                Dropout(dropout),
                Dense(1, activation='sigmoid')
            ])
            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                          loss='binary_crossentropy', metrics=['accuracy'])
            
            model.fit(X_train, y_train, validation_data=(X_val, y_val),
                      epochs=30, batch_size=256, sample_weight=weights,
                      callbacks=[EarlyStopping(patience=5, restore_best_weights=True)], verbose=0)
            
            # [수정] Accuracy -> ROC AUC
            y_pred_proba = model.predict(X_val, verbose=0).flatten()
            return roc_auc_score(y_val, y_pred_proba)

        study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
        study.optimize(objective, n_trials=5, show_progress_bar=False)

        p = study.best_params
        model = Sequential([
            GRU(p['units'], return_sequences=False, input_shape=input_shape,
                kernel_regularizer=l2(p['l2_reg']), dropout=p['dropout']),
            BatchNormalization(),
            Dense(16, activation='relu', kernel_regularizer=l2(p['l2_reg'])),
            Dropout(p['dropout']),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=p['learning_rate']),
                      loss='binary_crossentropy', metrics=['accuracy'])
        
        model.fit(X_train, y_train, validation_data=(X_val, y_val),
                  epochs=50, batch_size=256, sample_weight=weights,
                  callbacks=[EarlyStopping(patience=10, restore_best_weights=True)], verbose=0)
        
        y_pred_proba = model.predict(X_val, verbose=0).flatten()
        val_auc = roc_auc_score(y_val, y_pred_proba)
        print(f"[GRU] Val AUC: {val_auc:.4f}")
        return model


In [10]:
# ML Models (19개)
ML_MODELS_CLASSIFICATION = [
    {'index': 1, 'name': 'RandomForest', 'func': DirectionModels.random_forest, 'needs_val': True},
    {'index': 2, 'name': 'LightGBM', 'func': DirectionModels.lightgbm, 'needs_val': True},
    {'index': 3, 'name': 'XGBoost', 'func': DirectionModels.xgboost, 'needs_val': True},
    {'index': 5, 'name': 'LogisticRegression', 'func': DirectionModels.logistic_regression, 'needs_val': True},
    {'index': 8, 'name': 'AdaBoost', 'func': DirectionModels.adaboost, 'needs_val': True},
    {'index': 9, 'name': 'CatBoost', 'func': DirectionModels.catboost, 'needs_val': True},
    {'index': 13, 'name': 'GradientBoosting', 'func': DirectionModels.gradient_boosting, 'needs_val': True},
    {'index': 14, 'name': 'HistGradientBoosting', 'func': DirectionModels.histgradient_boosting, 'needs_val': True},
    {'index': 15, 'name': 'StackingEnsemble', 'func': DirectionModels.stacking_ensemble, 'needs_val': True},
]

# DL Models (11개)
DL_MODELS_CLASSIFICATION = [
    {'index': 19, 'name': 'LSTM', 'func': DirectionModels.lstm, 'needs_val': True},
   {'index': 20, 'name': 'BiLSTM', 'func': DirectionModels.bilstm, 'needs_val': True},
    {'index': 21, 'name': 'GRU', 'func': DirectionModels.gru, 'needs_val': True}
]


In [11]:
import numpy as np
import pandas as pd
import warnings
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score)

import numpy as np
import pandas as pd
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score)

class ModelEvaluator:
    def __init__(self, save_models=False):
        self.results = []
        self.predictions = {}
        self.models = {} if save_models else None
        self.save_models = save_models
    
    def _predict_proba_safe(self, model, X):
        if hasattr(model, 'predict_proba'):
            prob = model.predict_proba(X)
            return prob[:, 1] if prob.shape[1] == 2 else prob.ravel()
        else:
            try: return model.predict(X, verbose=0).ravel()
            except: return model.predict(X).ravel()

    def _calculate_metrics(self, y_true, pred_direction, prefix):
        y_true = np.array(y_true).ravel()
        pred_direction = np.array(pred_direction).ravel()
        valid_mask = (y_true != -1)
        if not valid_mask.any():
            return {f'{prefix}_Accuracy': 0.0, f'{prefix}_AUC_ROC': 0.5}
        
        y_true_v, pred_v = y_true[valid_mask], pred_direction[valid_mask]
        try: auc = roc_auc_score(y_true_v, pred_v)
        except: auc = 0.5
            
        return {
            f'{prefix}_Accuracy': accuracy_score(y_true_v, pred_v),
            f'{prefix}_Precision': precision_score(y_true_v, pred_v, zero_division=0),
            f'{prefix}_Recall': recall_score(y_true_v, pred_v, zero_division=0),
            f'{prefix}_F1': f1_score(y_true_v, pred_v, zero_division=0),
            f'{prefix}_AUC_ROC': auc
        }

    def _extract_y_details(self, y_data):
        n = len(y_data)
        direction = ret = entry_price = tp_price = sl_price = np.zeros(n)
        try:
            if hasattr(y_data, 'columns'):
                if 'next_direction' in y_data.columns: direction = y_data['next_direction'].values.ravel()
                if 'next_log_return' in y_data.columns: ret = y_data['next_log_return'].values.ravel()
                if 'next_open' in y_data.columns: entry_price = y_data['next_open'].values.ravel()
                if 'take_profit_price' in y_data.columns: tp_price = y_data['take_profit_price'].values.ravel()
                if 'stop_loss_price' in y_data.columns: sl_price = y_data['stop_loss_price'].values.ravel()
            else:
                direction = np.array(y_data).ravel()
        except: pass
        return direction, ret, entry_price, tp_price, sl_price

    def evaluate_classification_model(self, model, X_train, y_train, X_val, y_val, 
                                     X_test, y_test_df, train_dates, val_dates, test_dates,
                                     model_name, is_deep_learning=False):
        train_prob = self._predict_proba_safe(model, X_train)
        val_prob = self._predict_proba_safe(model, X_val)
        test_prob = self._predict_proba_safe(model, X_test)
        
        y_train_vals, _, _, _, _ = self._extract_y_details(y_train)
        y_val_vals, _, _, _, _ = self._extract_y_details(y_val)
        y_test_vals, _, _, _, _ = self._extract_y_details(y_test_df)

        metrics = {'Model': model_name}
        metrics.update(self._calculate_metrics(y_train_vals, (train_prob > 0.5).astype(int), 'Train'))
        metrics.update(self._calculate_metrics(y_val_vals, (val_prob > 0.5).astype(int), 'Val'))
        metrics.update(self._calculate_metrics(y_test_vals, (test_prob > 0.5).astype(int), 'Test'))
        
        self.results.append(metrics)
        if self.save_models and self.models is not None: self.models[model_name] = model
        return self.results[-1]

    def evaluate_with_calibration(self, model, X_train, y_train, X_val, y_val, 
                                 X_test, y_test_df, train_dates, val_dates, test_dates,
                                 model_name, tp_sl_ratio=2.0, 
                                 train_df=None, val_df=None):
        
        train_prob = self._predict_proba_safe(model, X_train)
        val_prob = self._predict_proba_safe(model, X_val)
        test_prob = self._predict_proba_safe(model, X_test)
        
        src_train = train_df if train_df is not None else y_train
        src_val = val_df if val_df is not None else y_val
        
        tr_dir, tr_ret, tr_open, tr_tp, tr_sl = self._extract_y_details(src_train)
        val_dir, val_ret, val_open, val_tp, val_sl = self._extract_y_details(src_val)
        test_dir, test_ret, test_open, test_tp, test_sl = self._extract_y_details(y_test_df)

        if train_df is not None and hasattr(y_train, 'values'): tr_dir = np.array(y_train).ravel()
        if val_df is not None and hasattr(y_val, 'values'): val_dir = np.array(y_val).ravel()

        best_threshold = 0.5
        best_score = -np.inf
        
        for th in np.arange(0.5, 0.85, 0.01):
            signals = (val_prob > th).astype(int)
            score = -np.inf if np.sum(signals) < 5 else np.sum(signals * val_ret)
            if score > best_score:
                best_score = score
                best_threshold = th
        
        train_signals_dyn = (train_prob > best_threshold).astype(int)
        val_signals_dyn = (val_prob > best_threshold).astype(int)
        test_signals_dyn = (test_prob > best_threshold).astype(int)
        
        # ✅ 마지막 행 업데이트 (새로운 행 추가하지 않음)
        metrics = self.results[-1]
        metrics.update(self._calculate_metrics(test_dir, test_signals_dyn, 'Test_Dynamic'))
        metrics.update(self._calculate_metrics(tr_dir, train_signals_dyn, 'Train_Dynamic'))
        metrics.update(self._calculate_metrics(val_dir, val_signals_dyn, 'Val_Dynamic'))
        metrics['Optimal_Threshold'] = best_threshold
        metrics['Test_Dynamic_Return'] = np.sum(test_signals_dyn * test_ret)
        metrics['Test_Dynamic_Trades'] = np.sum(test_signals_dyn)
        
        self._save_predictions(f"{model_name}_dynamic_train", train_dates, tr_dir, train_signals_dyn, train_prob, tr_ret, np.repeat(best_threshold, len(train_dates)), tr_open, tr_tp, tr_sl)
        self._save_predictions(f"{model_name}_dynamic_val", val_dates, val_dir, val_signals_dyn, val_prob, val_ret, np.repeat(best_threshold, len(val_dates)), val_open, val_tp, val_sl)
        self._save_predictions(f"{model_name}_dynamic_test", test_dates, test_dir, test_signals_dyn, test_prob, test_ret, np.repeat(best_threshold, len(test_dates)), test_open, test_tp, test_sl)
        
        return metrics

    def _save_predictions(self, name, dates, actual, pred, prob, ret, threshold, entry, tp, sl):
        min_len = min(len(dates), len(actual), len(pred))
        self.predictions[name] = pd.DataFrame({
            'date': dates[:min_len], 'actual': actual[:min_len], 'pred': pred[:min_len],
            'prob': prob[:min_len], 'return': ret[:min_len], 'threshold': threshold[:min_len],
            'entry_price': entry[:min_len], 'tp_price': tp[:min_len], 'sl_price': sl[:min_len]
        })

    def get_summary_dataframe(self): return pd.DataFrame(self.results)
    def get_predictions_dict(self): return self.predictions
    def get_models_dict(self): return self.models



In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
import gc
import traceback

# ==================================================================================
# [2] ModelTrainer: 모델 학습 및 Evaluator 호출 담당
# ==================================================================================
class ModelTrainer:
    def __init__(self, evaluator, lookback=30):
        self.evaluator = evaluator
        self.lookback = lookback
        self.tp_sl_ratio = 2.0

    def set_tp_sl_ratio(self, ratio):
        self.tp_sl_ratio = ratio
    
    @staticmethod
    def create_sequences(X, y, lookback):
        """
        X: Feature Matrix
        y: Target Array (Label)
        """
        Xs, ys = [], []
        y_vals = y.values if hasattr(y, 'values') else y
        
        # 길이가 부족하면 빈 배열 반환
        if len(X) <= lookback:
            return np.array([]), np.array([])

        for i in range(lookback, len(X)):
            Xs.append(X[i-lookback:i])
            ys.append(y_vals[i])
        return np.array(Xs), np.array(ys)
    
    @staticmethod
    def clear_memory():
        try:
            tf.keras.backend.clear_session()
        except:
            pass
        gc.collect()
        
    def train_ml_model(self, model_config, X_train, y_train_label, X_val, y_val_label, 
                       X_test, y_test_df, 
                       train_dates, val_dates, test_dates,
                       train_df=None, val_df=None): # <--- [수정] 전체 DF 추가 수신
        model = None
        try:
            # 1. 모델 학습 (학습에는 순수 라벨만 사용)
            if model_config.get('needs_val', False):
                model = model_config['func'](X_train, y_train_label, X_val, y_val_label)
            else:
                model = model_config['func'](X_train, y_train_label)
            
            # 2. 기본 평가 (Threshold 0.5)
            self.evaluator.evaluate_classification_model(
                model, X_train, y_train_label, X_val, y_val_label, X_test, y_test_df,
                train_dates, val_dates, test_dates, model_config['name']
            )
            
            # 3. [중요] Dynamic Threshold 평가 (가격 정보 저장을 위해 train_df, val_df 전달)
            self.evaluator.evaluate_with_calibration(
                model, X_train, y_train_label, X_val, y_val_label, X_test, y_test_df,
                train_dates, val_dates, test_dates, 
                model_config['name'], tp_sl_ratio=self.tp_sl_ratio,
                train_df=train_df, val_df=val_df  # <--- [수정] Evaluator에 전달
            )
            return True
        except Exception as e:
            print(f"    ⚠️ {model_config['name']} Failed: {str(e)}")
            traceback.print_exc()
            return False
        finally:
            if not self.evaluator.save_models:
                del model
            gc.collect()

    def train_dl_model(self, model_config, X_train_seq, y_train_seq, X_val_seq, y_val_seq, 
                       X_test_seq, y_test_df_seq, train_dates_seq, val_dates_seq, test_dates_seq, 
                       input_shape, 
                       train_df_seq=None, val_df_seq=None): # <--- [수정] 잘린 DF 수신
        model = None
        try:
            self.clear_memory()
            
            # 1. 모델 학습
            model = model_config['func'](X_train_seq, y_train_seq, X_val_seq, y_val_seq, input_shape)
            
            # 2. 기본 평가
            self.evaluator.evaluate_classification_model(
                model, X_train_seq, y_train_seq, X_val_seq, y_val_seq,
                X_test_seq, y_test_df_seq, train_dates_seq, val_dates_seq, test_dates_seq,
                model_config['name'], is_deep_learning=True
            )
            
            # 3. [중요] Dynamic Threshold 평가
            # DL 모델도 시퀀스에 맞춰 잘린 DataFrame(train_df_seq, val_df_seq)을 넘겨야 함
            self.evaluator.evaluate_with_calibration(
                model, X_train_seq, y_train_seq, X_val_seq, y_val_seq,
                X_test_seq, y_test_df_seq, train_dates_seq, val_dates_seq, test_dates_seq,
                model_config['name'], tp_sl_ratio=self.tp_sl_ratio,
                train_df=train_df_seq, val_df=val_df_seq # <--- [수정] 전달
            )
            return True
        except Exception as e:
            print(f"    ⚠️ {model_config['name']} Failed: {str(e)}")
            traceback.print_exc()
            return False
        finally:
            self.clear_memory()

# ==================================================================================
# [3] train_all_models: ML/DL 통합 학습 함수
# ==================================================================================
def train_all_models(X_train, y_train_full, X_val, y_val_full, X_test, y_test_df,
                     train_dates, val_dates, test_dates, evaluator, 
                     lookback=30, ml_models=None, dl_models=None, tp_sl_ratio=2.0):
    """
    y_train_full, y_val_full: 가격 정보가 포함된 전체 DataFrame이라고 가정합니다.
    (만약 Series라면 내부에서 처리되지만 가격 정보는 저장되지 않습니다.)
    """
    
    trainer = ModelTrainer(evaluator, lookback)
    trainer.set_tp_sl_ratio(tp_sl_ratio)

    # --- 데이터 분리 (Label vs DataFrame) ---
    # 학습에는 'next_direction' 컬럼(혹은 Series 자체)만 필요
    # 평가/저장에는 전체 DataFrame이 필요
    
    # 1. Train 데이터
    if hasattr(y_train_full, 'columns') and 'next_direction' in y_train_full.columns:
        y_train_label = y_train_full['next_direction']
        train_df = y_train_full
    else:
        y_train_label = y_train_full # Series or Array
        train_df = None # 가격 정보 없음

    # 2. Validation 데이터
    if hasattr(y_val_full, 'columns') and 'next_direction' in y_val_full.columns:
        y_val_label = y_val_full['next_direction']
        val_df = y_val_full
    else:
        y_val_label = y_val_full
        val_df = None

    # 3. Test 데이터 (y_test_df는 이미 DataFrame이라고 가정되어 있음)

    # ===========================
    # 1. ML 모델 학습
    # ===========================
    if ml_models:
        for model_config in ml_models:
            # ML 모델은 시퀀스 변환 없이 그대로 사용
            trainer.train_ml_model(
                model_config, 
                X_train, y_train_label, 
                X_val, y_val_label, 
                X_test, y_test_df, 
                train_dates, val_dates, test_dates,
                train_df=train_df, val_df=val_df # <--- [중요] 전체 DF 전달
            )
            gc.collect()
    
    # ===========================
    # 2. DL 모델 학습
    # ===========================
    if dl_models:
        trainer.clear_memory()
        
        # Sequence 생성 (Label 기준)
        X_train_seq, y_train_seq = trainer.create_sequences(X_train, y_train_label, lookback)
        X_val_seq, y_val_seq = trainer.create_sequences(X_val, y_val_label, lookback)
        
        # Test셋 Sequence 생성 (Test Label 추출)
        if hasattr(y_test_df, 'columns'):
            y_test_label = y_test_df['next_direction'].values
        else:
            y_test_label = y_test_df
            
        X_test_seq, y_test_seq = trainer.create_sequences(X_test, y_test_label, lookback)
        
        # 날짜 슬라이싱
        train_dates_seq = train_dates[lookback:]
        val_dates_seq = val_dates[lookback:]
        test_dates_seq = test_dates[lookback:]
        
        # [중요] DataFrame 슬라이싱 (DL은 앞부분 lookback만큼 데이터가 줄어듦)
        # reset_index(drop=True)를 해야 인덱스가 0부터 다시 시작해 나중에 합칠 때 문제없음
        train_df_seq = train_df.iloc[lookback:].reset_index(drop=True) if train_df is not None else None
        val_df_seq = val_df.iloc[lookback:].reset_index(drop=True) if val_df is not None else None
        y_test_df_seq = y_test_df.iloc[lookback:].reset_index(drop=True) if hasattr(y_test_df, 'iloc') else y_test_df
        
        input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])
        
        # DL 모델 루프
        for model_config in dl_models:
            trainer.train_dl_model(
                model_config, 
                X_train_seq, y_train_seq, 
                X_val_seq, y_val_seq, 
                X_test_seq, y_test_df_seq, 
                train_dates_seq, val_dates_seq, test_dates_seq, 
                input_shape,
                train_df_seq=train_df_seq, val_df_seq=val_df_seq # <--- [중요] 잘린 DF 전달
            )
            gc.collect()

    return True


In [13]:
import os
import pickle
import traceback
import glob
import gc
import numpy as np
import pandas as pd
import shap
import matplotlib.pyplot as plt

# 서버/CLI 환경에서 이미지 저장을 위해 백엔드 설정
plt.switch_backend('Agg')

def save_tree_shap_analysis(model, X_test, feature_names, output_dir, model_name):
    """
    모델별 SHAP 분석 및 저장 (Tree, Linear 지원, 예외 처리 강화)
    """
    import os
    import matplotlib
    matplotlib.use('Agg')
    import matplotlib.pyplot as plt
    import shap
    import pandas as pd
    import numpy as np
    import traceback
    
    # 1. 저장 경로 생성
    save_path = os.path.join(output_dir, "shap_analysis")
    os.makedirs(save_path, exist_ok=True)
    
    # 2. 데이터 전처리 및 샘플링
    try:
        if isinstance(X_test, pd.DataFrame):
            X_data = X_test.copy()
            if feature_names is None:
                feature_names = X_test.columns.tolist()
        elif isinstance(X_test, np.ndarray):
            if feature_names is None:
                feature_names = [f'feature_{i}' for i in range(X_test.shape[1])]
            X_data = pd.DataFrame(X_test, columns=feature_names)
        else:
            print(f"   -> SHAP Skip: Unknown X_test type: {type(X_test)}")
            return False
        
        # 속도 향상을 위한 샘플링 (최대 2000개)
        if len(X_data) > 2000:
            X_sample = X_data.sample(n=2000, random_state=42)
        else:
            X_sample = X_data.copy()
            
    except Exception as e:
        print(f"   -> SHAP Data Prep Error: {str(e)}")
        return False

    # 3. Explainer 선택 및 SHAP 값 계산
    explainer = None
    shap_values = None
    model_type = str(type(model))
    
    try:
        # (A) Tree 계열 (XGB, LGBM, CatBoost, RF, GB 등)
        if any(x in model_type for x in ['XGB', 'LGBM', 'CatBoost', 'RandomForest', 'GradientBoosting', 'DecisionTree', 'ExtraTrees']):
            try:
                explainer = shap.TreeExplainer(model)
                shap_values = explainer.shap_values(X_sample)
            except Exception as e:
                print(f"   -> TreeExplainer failed for {model_name}: {e}")
                return False

        # (B) Linear 계열 (LogisticRegression 등)
        elif 'LogisticRegression' in model_type or 'Linear' in model_type:
            try:
                explainer = shap.LinearExplainer(model, X_sample)
                shap_values = explainer.shap_values(X_sample)
            except Exception as e:
                print(f"   -> LinearExplainer failed for {model_name}: {e}")
                return False
        
        # (C) 기타 모델 (지원 안 함 또는 KernelExplainer 사용 시 너무 느림)
        else:
            print(f"   -> SHAP skipped for {model_name} (Type: {model_type} not optimized)")
            return False


        
        # Case 1: List 형태 (구버전 shap 또는 특정 모델)
        if isinstance(shap_values, list):
            if len(shap_values) > 1:
                shap_values = shap_values[1] # Class 1
            else:
                shap_values = shap_values[0]

        # Case 2: 3차원 Numpy Array (samples, features, classes) -> RandomForest가 주로 이럼
        elif isinstance(shap_values, np.ndarray) and len(shap_values.shape) == 3:
            # 마지막 차원이 클래스 개수일 때 (보통 2개)
            if shap_values.shape[2] >= 2:
                shap_values = shap_values[:, :, 1] # Class 1
            else:
                shap_values = shap_values[:, :, 0]

        # 5. CSV 저장 (차원 확인 후 생성)
        if shap_values is None or len(shap_values.shape) != 2:
             print(f"   -> SHAP Shape Error for {model_name}: {shap_values.shape if shap_values is not None else 'None'}")
             return False

        abs_shap = np.abs(shap_values)
        
        # DataFrame 생성 시 차원 불일치 에러 방지
        if abs_shap.shape[1] != len(feature_names):
             print(f"   -> SHAP Feature Mismatch: shap({abs_shap.shape[1]}) vs names({len(feature_names)})")
             # 이름 개수 맞추기 (임시 방편)
             if abs_shap.shape[1] < len(feature_names):
                 feature_names = feature_names[:abs_shap.shape[1]]
             else:
                 feature_names = [f"feat_{i}" for i in range(abs_shap.shape[1])]

        importance_df = pd.DataFrame({
            'feature': feature_names,
            'mean_abs_shap': abs_shap.mean(axis=0),
            'median_abs_shap': np.median(abs_shap, axis=0),
            'std_abs_shap': abs_shap.std(axis=0),
            'max_abs_shap': abs_shap.max(axis=0),
            'min_abs_shap': abs_shap.min(axis=0),
            'mean_shap': shap_values.mean(axis=0),
            'positive_ratio': (shap_values > 0).mean(axis=0)
        }).sort_values('mean_abs_shap', ascending=False)
        
        csv_path = os.path.join(save_path, f"{model_name}_shap_importance.csv")
        importance_df.to_csv(csv_path, index=False, float_format='%.6f')
        print(f"CSV saved: {csv_path}")
        
    except Exception as e:
        print(f"SHAP calculation failed for {model_name}: {str(e)}")
        # traceback.print_exc() # 디버깅 시 주석 해제
        return False
    
    # 6. 시각화 (Dot Plot & Bar Plot)
    if shap_values is not None and X_sample is not None:
        try:
            plt.figure(figsize=(10, 8))
            shap.summary_plot(
                shap_values, 
                X_sample, 
                feature_names=feature_names,
                show=False,
                max_display=20
            )
            plt.title(f"{model_name} SHAP Summary", fontsize=14, pad=20)
            plt.tight_layout()
            dot_path = os.path.join(save_path, f"{model_name}_summary_dot.png")
            plt.savefig(dot_path, dpi=300, bbox_inches='tight')
            plt.close()
            print(f"Dot plot saved: {dot_path}")
        except Exception as e:
            print(f"Dot plot failed: {str(e)}")
            plt.close('all')
        
        try:
            plt.figure(figsize=(10, 8))
            shap.summary_plot(
                shap_values, 
                X_sample, 
                feature_names=feature_names,
                plot_type="bar", 
                show=False,
                max_display=20
            )
            plt.title(f"{model_name} Feature Importance", fontsize=14, pad=20)
            plt.tight_layout()
            bar_path = os.path.join(save_path, f"{model_name}_importance_bar.png")
            plt.savefig(bar_path, dpi=300, bbox_inches='tight')
            plt.close()
            print(f"Bar plot saved: {bar_path}")
        except Exception as e:
            print(f"Bar plot failed: {str(e)}")
            plt.close('all')
    
    return True




def save_raw_data_once(result, target_name, split_method):
    raw_dir = os.path.join(RESULT_DIR, "raw_data", target_name, split_method)
    os.makedirs(raw_dir, exist_ok=True)
    
    for fold_idx, fold in enumerate(result, start=1):
        fold_type = fold['stats']['fold_type']
        fold_dir = os.path.join(raw_dir, f"fold_{fold_idx}_{fold_type}")
        os.makedirs(fold_dir, exist_ok=True)
        
        for split in ['train', 'val', 'test']:
            if 'X_raw' not in fold[split]:
                continue
                
            df = pd.DataFrame(fold[split]['X_raw'], columns=fold['stats']['selected_features'])
            df['date'] = fold[split]['dates']
            for col in fold[split]['y'].columns:
                df[col] = fold[split]['y'][col].values
            df.to_csv(os.path.join(fold_dir, f"{split}_raw.csv"), index=False, encoding='utf-8-sig')
        
        for split in ['train', 'val', 'test']:
            if 'X_raw' in fold[split]:
                del fold[split]['X_raw']
    
    gc.collect()

def check_fold_completed(target_name, fold_idx, fold_type):
    fold_dir = os.path.join(RESULT_DIR, "fold_results", target_name, f"fold_{fold_idx}_{fold_type}")
    
    required_files = [
        "fold_summary.csv",
        "selected_features.pkl",
        "inference_config.pkl"
    ]
    
    for file in required_files:
        if not os.path.exists(os.path.join(fold_dir, file)):
            return False
    
    model_files = [f for f in os.listdir(fold_dir) if f.endswith(('.pkl', '.h5')) 
                   and not any(x in f for x in ['scaler', 'features', 'stats', 'config'])]
    
    return len(model_files) > 0

def save_fold_results(fold_idx, fold_type, evaluator, target_name, fold_data):
    fold_dir = os.path.join(RESULT_DIR, "fold_results", target_name, f"fold_{fold_idx}_{fold_type}")
    os.makedirs(fold_dir, exist_ok=True)
    
    # --- SHAP 분석을 위한 데이터 준비 ---
    X_test_sample = fold_data['test']['X_robust']
    feature_names = fold_data['stats']['selected_features']
    
    # numpy array인 경우 DataFrame으로 변환해야 컬럼명이 그래프에 표시됨
    if not isinstance(X_test_sample, pd.DataFrame):
        X_test_sample = pd.DataFrame(X_test_sample, columns=feature_names)

    for model_name, model_obj in evaluator.get_models_dict().items():
        try:
            is_dl = isinstance(model_obj, tf.keras.Model)
            ext = ".h5" if is_dl else ".pkl"
            path = os.path.join(fold_dir, f"{model_name}{ext}")
            
            if is_dl: 
                model_obj.save(path)
            else:
                with open(path, 'wb') as f:
                    pickle.dump(model_obj, f)
                
                # [핵심 추가] 딥러닝이 아닌 모델(트리 모델 등)에 대해 SHAP 분석 수행
                save_tree_shap_analysis(model_obj, X_test_sample, feature_names, fold_dir, model_name)
                
        except Exception as e:
            print(f"Failed to save {model_name}: {e}")
    
    
    with open(os.path.join(fold_dir, "selected_features.pkl"), 'wb') as f:
        pickle.dump(fold_data['stats']['selected_features'], f)
    
    # missing_stats가 존재할 경우에만 저장
    if 'missing_stats' in fold_data['stats']:
        with open(os.path.join(fold_dir, "missing_stats.pkl"), 'wb') as f:
            pickle.dump(fold_data['stats']['missing_stats'], f)
    
    inference_config = {
        'selected_features': fold_data['stats']['selected_features'],
        'feature_order': fold_data['stats']['selected_features'], 
        'target_cols': fold_data['stats']['target_cols'],
        'target_type': fold_data['stats']['target_type'],
        'fold_type': fold_type,
        'fold_idx': fold_idx
    }
    with open(os.path.join(fold_dir, "inference_config.pkl"), 'wb') as f:
        pickle.dump(inference_config, f)
    
    for model_name, pred_df in evaluator.get_predictions_dict().items():
        pred_df.to_csv(os.path.join(fold_dir, f"{model_name}_predictions.csv"), 
                      index=False, encoding='utf-8-sig')
        
    summary = evaluator.get_summary_dataframe()
    summary.to_csv(os.path.join(fold_dir, "fold_summary.csv"), 
                   index=False, encoding='utf-8-sig')
    
    return summary, evaluator.get_predictions_dict()

def load_fold_results(target_name, fold_idx, fold_type):
    fold_dir = os.path.join(RESULT_DIR, "fold_results", target_name, f"fold_{fold_idx}_{fold_type}")
    summary_path = os.path.join(fold_dir, "fold_summary.csv")
    if not os.path.exists(summary_path): 
        return None, None
    summary = pd.read_csv(summary_path)
    predictions = {f.replace('_predictions.csv', ''): pd.read_csv(os.path.join(fold_dir, f)) 
                   for f in os.listdir(fold_dir) if f.endswith('_predictions.csv')}
    return summary, predictions

def save_walk_forward_summary(all_fold_results, target_name):
    if not all_fold_results: 
        return
    detailed_df = pd.concat([df.assign(Fold=i+1, fold_type=ft) 
                             for i, (df, ft) in enumerate(all_fold_results)], ignore_index=True)
    detailed_df.to_csv(os.path.join(RESULT_DIR, f"{target_name}_all_folds_detailed.csv"), 
                       index=False, encoding='utf-8-sig')
    
    wf_data = detailed_df[detailed_df['fold_type'] == 'walk_forward'].copy()
    if wf_data.empty: 
        return

    numeric_cols = wf_data.select_dtypes(include=np.number).columns.drop('Fold', errors='ignore')
    avg_results = wf_data.groupby('Model')[numeric_cols].agg(['mean', 'std']).reset_index()
    avg_results.columns = ['_'.join(col).strip() if col[1] else col[0] for col in avg_results.columns.values]
    if 'Test_Accuracy_mean' in avg_results.columns:
        avg_results = avg_results.sort_values(by='Test_Accuracy_mean', ascending=False).reset_index(drop=True)
    avg_results.to_csv(os.path.join(RESULT_DIR, f"{target_name}_walk_forward_average.csv"), 
                       index=False, encoding='utf-8-sig')

def run_and_save_master_summary(result_dir):
    try:
        summary_files = glob.glob(os.path.join(result_dir, "*_walk_forward_average.csv"))
        if not summary_files: 
            print("No walk_forward_average.csv files found")
            return

        master_list = []
        for f in summary_files:
            try:
                filename = os.path.basename(f)
                parts = filename.replace('direction_', '').replace('_walk_forward_average.csv', '').split('_')
                params = {p[0]: float(p[1:]) for p in parts}
                df = pd.read_csv(f)
                df['lookahead'] = params.get('l', np.nan)
                df['profit_mult'] = params.get('p', np.nan)
                df['stop_mult'] = params.get('s', np.nan)
                master_list.append(df)
            except Exception as e:
                print(f"Failed to process {f}: {e}")
                continue
            
        if not master_list:
            print("No valid summary files to combine")
            return
            
        master_df = pd.concat(master_list, ignore_index=True)
        if 'Test_Accuracy_mean' in master_df.columns:
            master_df = master_df.sort_values(by='Test_Accuracy_mean', ascending=False).reset_index(drop=True)
        
        master_df.to_csv(os.path.join(result_dir, "_MASTER_SUMMARY_RESULTS.csv"), 
                         index=False, encoding='utf-8-sig')
        print(f"Master summary saved: {len(master_list)} files combined")
        
    except Exception as e:
        print(f"Master summary failed: {e}")
        traceback.print_exc()

def check_experiment_completed(target_name, result_dir):
    summary_path = os.path.join(result_dir, f"{target_name}_walk_forward_average.csv")
    return os.path.exists(summary_path)



In [ ]:
import itertools
import os
import warnings
import traceback
import gc
import pandas as pd
import numpy as np
from datetime import datetime
from joblib import Parallel, delayed

# [중요] 메인 프로세스에서는 TensorFlow를 import 하지 않습니다.
# 대신 환경변수만 미리 설정하여, 자식 프로세스가 TF를 켤 때 설정을 먹도록 유도합니다.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

# ==================================================================================
# [1. 초기 설정]
# ==================================================================================
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# ==================================================================================
# [2. 데이터 로드 및 전처리]
# ==================================================================================
try:
    df_merged = pd.read_csv("eth_4_all.csv")
    
    # 날짜 변환 및 정렬
    df_merged['date'] = pd.to_datetime(df_merged['date'])
    df_merged = df_merged.sort_values('date').reset_index(drop=True)
    
    print(f"✅ 데이터 로드 및 날짜 변환 완료: {df_merged.shape}")
    print(f"   기간: {df_merged['date'].min()} ~ {df_merged['date'].max()}")
    
except FileNotFoundError:
    print("❌ 'eth_4hour_all.csv' 파일을 찾을 수 없습니다. 경로를 확인하세요.")
    raise FileNotFoundError("데이터 파일 없음")

timestamp = datetime.now().strftime("%Y-%m-%d")
RESULT_DIR = os.path.join("model_results", timestamp,"4시간봉1205")
os.makedirs(RESULT_DIR, exist_ok=True)

# ==================================================================================
# [3. 파라미터 설정]
# ==================================================================================


train_size_options = [4000, 6000]


lookahead_options = [12, 24 ,36]

atr_stop_multipliers = [ 2.5, 3.0]       
atr_profit_multipliers = [ 2.5, 3.5]
param_combinations = list(itertools.product(
    train_size_options,
    lookahead_options, 
    atr_profit_multipliers, 
    atr_stop_multipliers
))

print(f"🔥 총 실험 조합: {len(param_combinations)}개")
print(f"💾 결과 저장 경로: {RESULT_DIR}")


# ==================================================================================
# [4. 실험 실행 함수] 완벽하게 수정된 버전
# ==================================================================================
def run_daily_strategy_experiment(config):

    import tensorflow as tf
    import os
    import gc
    

    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        except RuntimeError as e:
            pass

    t_size, lookahead, p_mult, s_mult = config
    param_suffix = f"t{t_size}_l{lookahead}_p{p_mult}_s{s_mult}"
    current_target_name = f"direction_{param_suffix}"
    
    try:

        if check_experiment_completed(current_target_name, RESULT_DIR):
            return f"✓ SKIP: {param_suffix}"
        
        local_ml_models = [
            {'index': 1, 'name': 'RandomForest', 'func': DirectionModels.random_forest, 'needs_val': True},
            {'index': 2, 'name': 'LightGBM', 'func': DirectionModels.lightgbm, 'needs_val': True},
            {'index': 3, 'name': 'XGBoost', 'func': DirectionModels.xgboost, 'needs_val': True},
            {'index': 5, 'name': 'LogisticRegression', 'func': DirectionModels.logistic_regression, 'needs_val': True},
            {'index': 8, 'name': 'AdaBoost', 'func': DirectionModels.adaboost, 'needs_val': True},
            {'index': 9, 'name': 'CatBoost', 'func': DirectionModels.catboost, 'needs_val': True},
            {'index': 13, 'name': 'GradientBoosting', 'func': DirectionModels.gradient_boosting, 'needs_val': True},
            {'index': 14, 'name': 'HistGradientBoosting', 'func': DirectionModels.histgradient_boosting, 'needs_val': True},
            {'index': 15, 'name': 'StackingEnsemble', 'func': DirectionModels.stacking_ensemble, 'needs_val': True},
        ]

        local_dl_models = [
            {'index': 19, 'name': 'LSTM', 'func': DirectionModels.lstm, 'needs_val': True},
            {'index': 20, 'name': 'BiLSTM', 'func': DirectionModels.bilstm, 'needs_val': True},
            {'index': 21, 'name': 'GRU', 'func': DirectionModels.gru, 'needs_val': True}
        ]
        
        test_period_candles = 540  # 3개월
        
        wf_params = {
            'initial_train_size': t_size,
            
            'val_size': test_period_candles,   # 검증도 3개월치로 넉넉하게
            'test_size': test_period_candles,  # 테스트도 3개월치
            'step': test_period_candles,       # 3개월마다 모델 업데이트 (재학습 빈도 조절)
            
            'gap_size': lookahead + 1 # (여유분)
        }
        pipeline_result = build_complete_pipeline_corrected(
            df_merged, 
            train_start_date='2024-01-01',
            method='walk_forward',
            target_type='direction',
            final_test_start='2025-01-01',
            lookahead_candles=lookahead,
            atr_multiplier_profit=p_mult,
            atr_multiplier_stop=s_mult,
            **wf_params
        )
        
        if not pipeline_result or len(pipeline_result) == 0:
            return f"⚠️ SKIP (No Data): {param_suffix}"

        save_raw_data_once(pipeline_result, current_target_name, 'walk_forward')
        
        fold_results = []
        
        for fold_idx, fold_data in enumerate(pipeline_result, start=1):
            fold_type = fold_data['stats']['fold_type']
            
            if check_fold_completed(current_target_name, fold_idx, fold_type):
                fold_summary, _ = load_fold_results(current_target_name, fold_idx, fold_type)
                if fold_summary is not None:
                    fold_results.append((fold_summary, fold_type))
                continue
            
            # [메모리 정리 1]
            tf.keras.backend.clear_session()
            gc.collect()
            
            evaluator = ModelEvaluator(save_models=True)
            print(fold_data['test']['y']['next_direction'])
            try:
                train_all_models(
                    fold_data['train']['X_robust'], 
                    fold_data['train']['y']['next_direction'],
                    fold_data['val']['X_robust'], 
                    fold_data['val']['y']['next_direction'],
                    fold_data['test']['X_robust'], 
                    fold_data['test']['y'], 
                    fold_data['train']['dates'].values,
                    fold_data['val']['dates'].values,
                    fold_data['test']['dates'].values,
                    evaluator, 
                    ml_models=local_ml_models, 
                    dl_models=local_dl_models
                )
                
                
                fold_summary, _ = save_fold_results(
                    fold_idx, fold_type, evaluator, 
                    current_target_name, fold_data
                )
                fold_results.append((fold_summary, fold_type))
                
            except Exception as e:
                with open(os.path.join(RESULT_DIR, "fold_errors.log"), "a") as f:
                    f.write(f"{param_suffix} - Fold {fold_idx}: {str(e)}\n")
                    f.write(traceback.format_exc() + "\n")
                continue
                
            finally:
                # [메모리 정리 2] 평가 객체 삭제 및 세션 클리어
                del evaluator
                tf.keras.backend.clear_session()
                gc.collect()
        
        save_walk_forward_summary(fold_results, current_target_name)
        return f"✅ DONE: {param_suffix}"
        
    except Exception as e:
        error_msg = f"{param_suffix}: {str(e)}"
        with open(os.path.join(RESULT_DIR, "experiment_errors.log"), "a") as f:
            f.write(error_msg + "\n")
            f.write(traceback.format_exc() + "\n")
        return f"❌ FAIL: {param_suffix}"

# ==================================================================================
# [5. 메인 실행부]
# ==================================================================================
if __name__ == "__main__":
    N_JOBS = 1 
    
    print(f"🚀 병렬 처리 시작 (RTX 6000, n_jobs={N_JOBS})")

    results = Parallel(n_jobs=N_JOBS, backend='loky', verbose=10)(
        delayed(run_daily_strategy_experiment)(cfg) for cfg in param_combinations
    )
    
    run_and_save_master_summary(RESULT_DIR)

    print(f"\n🏁 모든 실험 종료 ({len(results)}개)")
    print(f"📝 최종 결과 파일: {os.path.join(RESULT_DIR, '_MASTER_SUMMARY_RESULTS.csv')}")


✅ 데이터 로드 및 날짜 변환 완료: (17936, 181)
   기간: 2017-09-26 17:00:00 ~ 2025-12-03 09:00:00
🔥 총 실험 조합: 24개
💾 결과 저장 경로: model_results/2025-12-05/4시간봉1205
🚀 병렬 처리 시작 (RTX 6000, n_jobs=1)
=== 정답(Label) 분포 확인 ===
0.0    0.583831
1.0    0.416169
Name: next_direction, dtype: float64
Saved raw data to model_results/2025-12-05/raw_data
Removing 41 raw price columns
Not enough data. Total: 4203, Required: 5096

Reverse Rolling Walk-Forward Configuration 
Total: 4203 days
Rolling train size: 4000 days (FIXED)
Val: 540 days | Test: 540 days
Gap: 8 days | Step: 540 days (BACKWARD)
Target: 0 walk-forward + 1 final holdout

Fold 1 (final_holdout)
  Train: 1648d  2024-01-01 ~ 2024-10-01
  Val:    540d  2024-10-03 ~ 2024-12-31
  Test:  2007d  2025-01-01 ~ 2025-12-01

Created 1 folds total


Processing Fold 1 (final_holdout)
[Pre-cleaning] Dropped rows with NaN targets (Leakage prevention):
  Train: 1648 -> 1648 (Dropped 0)
  Val:   540 -> 540 (Dropped 0)
  Test:  2007 -> 2006 (Dropped 1)

[Feature Selection fo

[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    7.3s


=== 정답(Label) 분포 확인 ===
0.0    0.577805
1.0    0.422195
Name: next_direction, dtype: float64
Saved raw data to model_results/2025-12-05/raw_data
Removing 41 raw price columns
Not enough data. Total: 4203, Required: 5096

Reverse Rolling Walk-Forward Configuration 
Total: 4203 days
Rolling train size: 4000 days (FIXED)
Val: 540 days | Test: 540 days
Gap: 8 days | Step: 540 days (BACKWARD)
Target: 0 walk-forward + 1 final holdout

Fold 1 (final_holdout)
  Train: 1648d  2024-01-01 ~ 2024-10-01
  Val:    540d  2024-10-03 ~ 2024-12-31
  Test:  2007d  2025-01-01 ~ 2025-12-01

Created 1 folds total


Processing Fold 1 (final_holdout)
[Pre-cleaning] Dropped rows with NaN targets (Leakage prevention):
  Train: 1648 -> 1648 (Dropped 0)
  Val:   540 -> 540 (Dropped 0)
  Test:  2007 -> 2006 (Dropped 1)

[Feature Selection for Fold 1]
Training data shape: (1648, 140)
[Clustering] 140개 -> 64개로 압축 (중복 제거 완료)
최종 선택된 피처(30개): fred_RRPONTSYD_trend_60d, fred_DTWEXBGS_trend_20d, fred_DTWEXBGS_diff_1d, fre

[Parallel(n_jobs=1)]: Done   4 tasks      | elapsed:   26.2s


=== 정답(Label) 분포 확인 ===
0.0    0.532131
1.0    0.467869
Name: next_direction, dtype: float64
Saved raw data to model_results/2025-12-05/raw_data
Removing 41 raw price columns
Not enough data. Total: 4191, Required: 5096

Reverse Rolling Walk-Forward Configuration 
Total: 4191 days
Rolling train size: 4000 days (FIXED)
Val: 540 days | Test: 540 days
Gap: 8 days | Step: 540 days (BACKWARD)
Target: 0 walk-forward + 1 final holdout

Fold 1 (final_holdout)
  Train: 1648d  2024-01-01 ~ 2024-10-01
  Val:    540d  2024-10-03 ~ 2024-12-31
  Test:  1995d  2025-01-01 ~ 2025-11-29

Created 1 folds total


Processing Fold 1 (final_holdout)
[Pre-cleaning] Dropped rows with NaN targets (Leakage prevention):
  Train: 1648 -> 1648 (Dropped 0)
  Val:   540 -> 540 (Dropped 0)
  Test:  1995 -> 1994 (Dropped 1)

[Feature Selection for Fold 1]
Training data shape: (1648, 140)
[Clustering] 140개 -> 64개로 압축 (중복 제거 완료)
최종 선택된 피처(30개): fred_DTWEXBGS_trend_20d, fred_WALCL_diff_1d, ETH_Price_div_VWMA_50d, eth_chai

...

Bar plot saved: model_results/2025-12-05/4시간봉1205/fold_results/direction_t6000_l12_p3.5_s2.5/fold_1_final_holdout/shap_analysis/XGBoost_importance_bar.png
CSV saved: model_results/2025-12-05/4시간봉1205/fold_results/direction_t6000_l12_p3.5_s2.5/fold_1_final_holdout/shap_analysis/LogisticRegression_shap_importance.csv
Dot plot saved: model_results/2025-12-05/4시간봉1205/fold_results/direction_t6000_l12_p3.5_s2.5/fold_1_final_holdout/shap_analysis/LogisticRegression_summary_dot.png
Bar plot saved: model_results/2025-12-05/4시간봉1205/fold_results/direction_t6000_l12_p3.5_s2.5/fold_1_final_holdout/shap_analysis/LogisticRegression_importance_bar.png
   -> SHAP skipped for AdaBoost (Type: <class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'> not optimized)
CSV saved: model_results/2025-12-05/4시간봉1205/fold_results/direction_t6000_l12_p3.5_s2.5/fold_1_final_holdout/shap_analysis/CatBoost_shap_importance.csv
Dot plot saved: model_results/2025-12-05/4시간봉1205/fold_results/direction_t6000_l12_p3

Dot plot saved: model_results/2025-12-05/4시간봉1205/fold_results/direction_t6000_l12_p3.5_s3.0/fold_1_final_holdout/shap_analysis/HistGradientBoosting_summary_dot.png
Bar plot saved: model_results/2025-12-05/4시간봉1205/fold_results/direction_t6000_l12_p3.5_s3.0/fold_1_final_holdout/shap_analysis/HistGradientBoosting_importance_bar.png
   -> SHAP skipped for StackingEnsemble (Type: <class 'sklearn.ensemble._stacking.StackingClassifier'> not optimized)
=== 정답(Label) 분포 확인 ===
0.0    0.532131
1.0    0.467869
Name: next_direction, dtype: float64
Saved raw data to model_results/2025-12-05/raw_data
Removing 41 raw price columns
Not enough data. Total: 4191, Required: 7096

Reverse Rolling Walk-Forward Configuration 
Total: 4191 days
Rolling train size: 6000 days (FIXED)
Val: 540 days | Test: 540 days
Gap: 8 days | Step: 540 days (BACKWARD)
Target: 0 walk-forward + 1 final holdout

Fold 1 (final_holdout)
  Train: 1648d  2024-01-01 ~ 2024-10-01
  Val:    540d  2024-10-03 ~ 2024-12-31
  Test:  1995

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed: 37.2min


=== 정답(Label) 분포 확인 ===
0.0    0.515158
1.0    0.484842
Name: next_direction, dtype: float64
Saved raw data to model_results/2025-12-05/raw_data
Removing 41 raw price columns
Not enough data. Total: 4191, Required: 7096

Reverse Rolling Walk-Forward Configuration 
Total: 4191 days
Rolling train size: 6000 days (FIXED)
Val: 540 days | Test: 540 days
Gap: 8 days | Step: 540 days (BACKWARD)
Target: 0 walk-forward + 1 final holdout

Fold 1 (final_holdout)
  Train: 1648d  2024-01-01 ~ 2024-10-01
  Val:    540d  2024-10-03 ~ 2024-12-31
  Test:  1995d  2025-01-01 ~ 2025-11-29

Created 1 folds total


Processing Fold 1 (final_holdout)
[Pre-cleaning] Dropped rows with NaN targets (Leakage prevention):
  Train: 1648 -> 1648 (Dropped 0)
  Val:   540 -> 540 (Dropped 0)
  Test:  1995 -> 1994 (Dropped 1)

[Feature Selection for Fold 1]
Training data shape: (1648, 140)
[Clustering] 140개 -> 64개로 압축 (중복 제거 완료)
최종 선택된 피처(30개): fred_RRPONTSYD_diff_1d, fred_RRPONTSYD_trend_60d, fred_DTWEXBGS_trend_60d, fr

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
1989    0.0
1990    0.0
1991    0.0
1992    0.0
1993    0.0
Name: next_direction, Length: 1994, dtype: float64
[RandomForest] Val AUC: 0.5768
[LightGBM] Val AUC: 0.5792
[XGBoost] Val AUC: 0.5495
[Logistic Regression] Val AUC: 0.5815
[AdaBoost] Val AUC: 0.5784
[CatBoost] Val AUC: 0.5859
[GradientBoosting] Val AUC: 0.5545
[HistGradientBoosting] Val AUC: 0.5740
[Stacking] Val AUC: 0.4281
[LSTM] Val AUC: 0.4328
[BiLSTM] Val AUC: 0.5776
[GRU] Val AUC: 0.3051
CSV saved: model_results/2025-12-05/4시간봉1205/fold_results/direction_t6000_l24_p3.5_s2.5/fold_1_final_holdout/shap_analysis/RandomForest_shap_importance.csv
Dot plot saved: model_results/2025-12-05/4시간봉1205/fold_results/direction_t6000_l24_p3.5_s2.5/fold_1_final_holdout/shap_analysis/RandomForest_summary_dot.png
Bar plot saved: model_results/2025-12-05/4시간봉1205/fold_results/direction_t6000_l24_p3.5_s2.5/fold_1_final_holdout/shap_analysis/RandomForest_importance_bar.p

In [ ]:
! nvidia-smi